In [1]:
import numpy as np

import chromadb
from chromadb.utils import embedding_functions

from chromadb.config import Settings

chroma_client = chromadb.HttpClient(host='localhost', port = 8083, settings=Settings(allow_reset=True, anonymized_telemetry=False))

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

collection = chroma_client.get_or_create_collection(name="transcripts_mililm_l6_v3", embedding_function=sentence_transformer_ef)


/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
#query one document
results = collection.query(
    query_texts=["Tesla transcripts for the year 2024"],
    n_results=10,
    where = {'$and':[
              {'symbol': {
                       "$in": ['TSLA']}
              }, 
              {'year': {
                        "$gt": 2023}
              }]
         },
    include=['metadatas', 'documents', 'embeddings'],
    where_document={"$contains":"2024"}
)

In [3]:
results

{'ids': [['2024Q2TSLA', '2024Q1TSLA']],
 'distances': None,
 'embeddings': [[[-0.0822194516658783,
    0.009658602997660637,
    0.052224915474653244,
    0.02644718624651432,
    0.04375709965825081,
    -0.012698511593043804,
    -0.000993350287899375,
    0.07763923704624176,
    0.046971507370471954,
    -0.026099272072315216,
    0.003186175599694252,
    0.0008456492214463651,
    0.04624375328421593,
    -0.07265063375234604,
    0.011303565464913845,
    0.02751617692410946,
    0.0003328380116727203,
    -0.1211201548576355,
    -0.06751690059900284,
    0.05124247446656227,
    -0.03266194090247154,
    -0.013144985772669315,
    -0.0125283133238554,
    0.002807856537401676,
    0.04040118679404259,
    -0.01640661247074604,
    -0.03554452955722809,
    0.05132472142577171,
    -0.026458900421857834,
    -0.032147157937288284,
    -0.019565070047974586,
    0.045829273760318756,
    0.03245055675506592,
    0.0004907945403829217,
    -0.02981671132147312,
    0.030537785962

In [4]:
documents = ' '.join(results['documents'][0])

In [5]:
documents

"Travis Axelrod: Good afternoon, everyone and welcome to Tesla's Second Quarter 2024 Q&A Webcast. My name is Travis Axelrod, Head of Investor Relations and I’m joined today by Elon Musk, Vaibhav Taneja, and a number of other executives. Our Q2 results were announced at about 3.00 p.m. Central Time and the Update Deck we published at the same link as this webcast. During this call, we will discuss our business outlook and make forward-looking statements. These comments are based on our predictions and expectations as of today. Actual events or results could differ materially due to a number of risks and uncertainties, including those mentioned in our most recent filings with the SEC. During the question-and-answer portion of today's call, please limit yourself to one question and one follow-up. Please use the raise hand button to join the question queue. Before we jump into Q&A, Elon has some opening remarks. Elon?\nElon Musk: Thank you. So to recap, we saw large adoption exploration in

In [6]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)

texts = text_splitter.create_documents([documents])
split_texts = text_splitter.split_documents(texts)


In [40]:
split_ls = [x.page_content for x in split_texts]
len(split_ls)

138

In [42]:
%time

from transformers import pipeline
from tqdm import tqdm

import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"


summ_ls = []

for sub in tqdm(split_ls):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device='mps')
    sum = summarizer(sub, max_length=150, min_length=30, do_sample=False)
    summ_ls.append(sum)

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 2.86 µs


  0%|          | 0/138 [00:00<?, ?it/s]/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
  1%|          | 1/138 [00:25<57:21, 25.12s/it]


KeyboardInterrupt: 

In [102]:
summ_ls

[[{'summary_text': "This conference call is being recorded and will be available for replay from the Investor Relations section of Airbnb's website following this call. I will now hand the call over to Elli Mertz, VP of Finance. Please go ahead."}],
 [{'summary_text': "Ellie Mertz: Thank you. Good afternoon and welcome to Airbnb's third quarter of 2023 earnings call. On the call today, we have Airbnb's Co-Founder and CEO, Brian Chesky and our Chief Financial Officer, Dave Stephenson. During the call, we'll make brief opening remarks and then spend the remainder of time on Q&A."}],
 [{'summary_text': 'During this call, we will discuss some non-GAAP financial measures. These measures are not intended to be a substitute for our GAAP results. You should be aware that these statements should be considered estimates only.'}],
 [{'summary_text': 'Brian Chesky: Q3 was another strong quarter for Airbnb. We had over 113 million Nights and Experiences Booked. Revenue of $3.4 billion grew 18% year

In [106]:
summarized_text = " ".join([x[0]['summary_text'] for x in summ_ls])

In [110]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)

texts = text_splitter.create_documents([summarized_text])
split_texts = text_splitter.split_documents(texts)

In [7]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.llm import LLMChain

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate
local_path = (
    "../llm_models/gpt4all-falcon-q4_0.gguf"  # replace with your desired local file path
)

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

In [47]:
from mlx_lm import load

# Define your model to import
model_name = "mlx-community/Meta-Llama-3-8B-Instruct-4bit"

# Loading model
model, tokenizer = load(model_name)

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 93553.25it/s]


In [46]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

model_id = "meta-llama/Meta-Llama-3-8B"

llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="mps",)

KeyboardInterrupt: 

In [8]:
from langchain_core.prompts import PromptTemplate

from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="refine", verbose=True)

In [9]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain

# Map
map_template = """The following is a set of summarized transcripts
{docs}
Based on this list of docs, generate a summary of the main points discussed.
Don't return conversation between people, just distill the main points.
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

In [10]:
# Reduce
reduce_template = """The following is set of summaries of transcripts:
{docs}
Take these and distill it into a final, consolidated summary of the main challenges and successes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [11]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain


# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=2000,
)

In [12]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False)

In [13]:
%time
result = map_reduce_chain.run(split_texts)

Exception ignored on calling ctypes callback function: <function LLModel._prompt_callback at 0x2c0bfab00>
Traceback (most recent call last):
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 479, in _prompt_callback
    @staticmethod
KeyboardInterrupt: 




1. Tesla's Q2 2024 results were announced at around 3:00 p.m. Central Time.
2. The Update Deck was published at the same link as this webcast.
3. During this call, the company will discuss its business outlook and make forward-looking statements.
4. Actual events or results could differ materially due to a number of risks and uncertainties, including those mentioned in the company's most recent filings with the SEC.
5. During the question-and-answer portion of today's call, please limit yourself to one question and one follow-up.
6. Please use the raise hand button to join the question queue.
7. Before we jump into Q&A, Elon Musk will make some opening remarks.
8. The company's Q2 2024 results were announced at around 3:00 p.m. Central Time.
9. The Update Deck was published at the same link as this webcast.
10. During this call, the company will discuss its business outlook and make forward-looking statements.
11. Actual events or results could differ materially due to a number of ri

Exception ignored on calling ctypes callback function: <function LLModel._callback_decoder.<locals>._raw_callback at 0x2c0bfb760>
Traceback (most recent call last):
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/gpt4all/pyllmodel.py", line 438, in _raw_callback
    def _raw_callback(token_id: int, response: bytes) -> bool:
KeyboardInterrupt: 




The main points discussed in the list of documents are:

* Tesla achieved record quarterly revenues and profits for the energy business in Q2 2023.
* The company is investing in many future projects, including AI training and inference, and infrastructure to support future products.
* Tesla's CEO, Elon Musk, believes that EVs are best for customers and that the world is headed for a fully electrified transport, not just the cars, but also aircrafts and boats.


: 

: 

In [122]:
result

"\nAirbnb's main challenges include increasing supply and improving search experience, while its main successes include focusing on home rentals, innovating on the platform, and expanding into new markets. The company is also exploring opportunities in other areas like vacation rentals and experiences."